In [2]:
import torch
import random
import numpy as np
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import pandas as pd

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [14]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
LABELS = [    
    "JUMPING",
    "JUMPING_JACKS",
    "BOXING",
    "WAVING_2HANDS",
    "WAVING_1HAND",
    "CLAPPING_HANDS"

] 
DATASET_PATH = "poses/"

X_train_path = "shuffled_x_train_data"
X_test_path = DATASET_PATH + "X_test.txt"

y_train_path = "shuffled_y_train_data"
y_test_path = DATASET_PATH + "Y_test.txt"

# Hyper-parameters 
# input_size = 784 # 28x28
num_classes = 6
num_epochs = 10
train_batch_size = 125
test_batch_size = 1
learning_rate = 0.001

input_size = 36
sequence_length = 32
hidden_size = 128
num_layers = 2

In [8]:
device

device(type='cpu')

In [15]:
column_names = [  "j0_x",  "j0_y", "j1_x", "j1_y" , "j2_x", "j2_y", "j3_x", "j3_y", "j4_x", "j4_y", "j5_x", "j5_y", "j6_x", "j6_y", "j7_x", "j7_y", "j8_x", "j8_y", "j9_x", "j9_y", "j10_x", "j10_y", "j11_x", "j11_y", "j12_x", "j12_y", "j13_x", "j13_y", 'j14_x', "j14_y", "j15_x", "j15_y", "j16_x", "j16_y", "j17_x", "j17_y" ]
# x_train_data = pd.read_csv(X_train_path, sep=",", names=column_names, header=None, dtype=np.float32)
x_train_data = pd.read_csv(X_train_path, sep="\t", dtype=np.float32)
x_test_data = pd.read_csv(X_test_path, sep=",", names=column_names, header=None, dtype=np.float32)
# y_train_data = p|d.read_csv(y_train_path, names=["labels"], dtype=np.int_)
y_train_data = pd.read_csv(y_train_path, sep="\t", dtype=np.int_)
y_test_data = pd.read_csv(y_test_path, names=["labels"], dtype=np.int_)

In [18]:
shuffled_x_train_data = x_train_data
shuffled_y_train_data = y_train_data

In [20]:
def normalize(data):
    for i, column in enumerate(data):
        if i % 2 == 0:
            data[column] = data[column] / 640
        else:
            data[column] = data[column] / 480
    return data

In [21]:
# normalize(x_train_data)
normalize(x_test_data).head()


,j0_x,j0_y,j1_x,j1_y,j2_x,j2_y,j3_x,j3_y,j4_x,j4_y,...,j13_x,j13_y,j14_x,j14_y,j15_x,j15_y,j16_x,j16_y,j17_x,j17_y
0,0.480608,0.339533,0.499006,0.429050,0.458230,0.426417,0.445991,0.521419,0.433775,0.597352,...,0.527416,0.858235,0.476553,0.333940,0.492905,0.333894,0.0,0.0,0.515238,0.336773
1,0.480573,0.339540,0.499003,0.429056,0.458214,0.426448,0.445981,0.521417,0.433775,0.594800,...,0.527414,0.858202,0.476514,0.333948,0.492866,0.333892,0.0,0.0,0.513323,0.336781
2,0.478591,0.339481,0.498986,0.429035,0.456236,0.426415,0.445988,0.524065,0.433777,0.597444,...,0.527422,0.858233,0.472530,0.333890,0.490823,0.333875,0.0,0.0,0.513273,0.336835
3,0.478508,0.336883,0.498959,0.426423,0.456230,0.426333,0.445983,0.521427,0.431887,0.597506,...,0.527445,0.858206,0.470505,0.331300,0.488778,0.331308,0.0,0.0,0.513252,0.336785
4,0.476562,0.336775,0.497027,0.426348,0.454209,0.423727,0.445997,0.521408,0.431880,0.597565,...,0.527423,0.858217,0.470452,0.328621,0.486744,0.331202,0.0,0.0,0.513173,0.336777


In [22]:
# y_train_data = y_train_data - 1
y_test_data = y_test_data - 1
y_test_data

,labels
0,0
1,0
2,0
3,0
4,0
...,...
5746,5
5747,5
5748,5
5749,5


In [113]:
shuffled_y_train_data = pd.DataFrame()
shuffled_y_train_data = shuffled_y_train_data.append(x_train_data.iloc[0:32], ignore_index=True)
shuffled_y_train_data = shuffled_y_train_data.append(x_train_data.iloc[1*32:2*64], ignore_index=True)
shuffled_y_train_data

,j0_x,j0_y,j1_x,j1_y,j2_x,j2_y,j3_x,j3_y,j4_x,j4_y,...,j13_x,j13_y,j14_x,j14_y,j15_x,j15_y,j16_x,j16_y,j17_x,j17_y
0,0.462366,0.336623,0.480770,0.423777,0.439916,0.423683,0.429683,0.524087,0.417491,0.610944,...,0.513325,0.858171,0.460228,0.325875,0.476566,0.325871,0.000000,0.000000,0.497005,0.336733
1,0.462273,0.336667,0.480756,0.423767,0.439889,0.423719,0.429670,0.524113,0.417486,0.608252,...,0.513270,0.855479,0.458206,0.328513,0.474541,0.328554,0.000000,0.000000,0.496912,0.336779
2,0.458275,0.336708,0.480711,0.423771,0.437856,0.423713,0.429666,0.524148,0.417597,0.602860,...,0.513316,0.858221,0.452147,0.328513,0.468434,0.328571,0.000000,0.000000,0.494963,0.336831
3,0.456216,0.336788,0.480655,0.426412,0.435809,0.426373,0.429689,0.529404,0.421553,0.602840,...,0.513248,0.858223,0.450158,0.328598,0.466386,0.331219,0.000000,0.000000,0.494875,0.336869
4,0.450098,0.350250,0.478691,0.437156,0.431850,0.439808,0.429703,0.532210,0.429692,0.613710,...,0.515272,0.858092,0.448084,0.336658,0.464270,0.336669,0.000000,0.000000,0.492816,0.347583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,0.482669,0.461650,0.507156,0.543058,0.462413,0.537667,0.456241,0.621944,0.448055,0.700533,...,0.539755,0.836385,0.478620,0.448131,0.494927,0.445487,0.466450,0.467079,0.519323,0.464240
125,0.482670,0.458877,0.509088,0.537698,0.464319,0.535040,0.454194,0.624552,0.445911,0.700531,...,0.539711,0.833735,0.478647,0.445417,0.494953,0.445260,0.472498,0.467006,0.521408,0.458910
126,0.480758,0.434465,0.507131,0.513310,0.464323,0.513231,0.454150,0.597542,0.437884,0.665379,...,0.539739,0.833608,0.480572,0.421023,0.494912,0.423521,0.474497,0.442490,0.519367,0.437152
127,0.482702,0.399183,0.511097,0.486019,0.466362,0.485990,0.448037,0.575627,0.427670,0.640977,...,0.545772,0.836533,0.480677,0.388329,0.494923,0.385629,0.000000,0.000000,0.517366,0.399210


In [120]:
from random import shuffle

# def shuffle_data(x_train_data, y_train_data):
length = y_train_data.shape[0]
shuffled_indexes = [i for i in range(length)]
shuffle(shuffled_indexes)
shuffled_y_train_data = pd.DataFrame()
shuffled_x_train_data = pd.DataFrame()
for i, index in enumerate(shuffled_indexes): 
    shuffled_y_train_data = shuffled_y_train_data.append(y_train_data.iloc[[index]], ignore_index=True)
    shuffled_x_train_data = shuffled_x_train_data.append(x_train_data.iloc[index*32:index*32+32], ignore_index=True)
    if (i % 1000 == 0):
        print(f"{i} done")

0 done
1000 done
2000 done
3000 done
4000 done
5000 done
6000 done
7000 done
8000 done
9000 done
10000 done
11000 done
12000 done
13000 done
14000 done
15000 done
16000 done
17000 done
18000 done
19000 done
20000 done
21000 done
22000 done


In [123]:
shuffled_x_train_data.to_csv("shuffled_x_train_data", sep='\t', index=False)
shuffled_y_train_data.to_csv("shuffled_y_train_data", sep='\t', index=False)

In [23]:
shuffled_x_train_data.shape

(724000, 36)

In [24]:
# Fully connected neural network with one hidden layer
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTM, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        # -> x needs to be: (batch_size, seq, input_size)
        
        # or:100
        #self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        #self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        # Set initial hidden states (and cell states for LSTM)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        
        # x: (n, 28, 28), h0: (2, n, 128)
        
        # Forward propagate RNN
        # out, _ = self.rnn(x, h0)  
        # or:
        out, _ = self.lstm(x, (h0,c0))  
        
        # out: tensor of shape (batch_size, seq_length, hidden_size)
        # out: (n, 28, 128)
        
        # Decode the hidden state of the last time step
        out = out[:, -1, :]
        # out: (n, 128)
         
        out = self.fc(out)
        # out: (n, 10)
        return out

In [25]:
model = LSTM(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.01)  

In [26]:
def chunker(x_seq, y_seq, seq_size, batch_size):
    for batch_pos in range(0, len(y_seq), batch_size):
        x_batch = list()
        for pos in range(batch_size):
            # print(x_seq.iloc[pos*seq_size:pos*seq_size + seq_size])
            x_batch.append(x_seq.iloc[pos*seq_size:pos*seq_size + seq_size].values)
        
        yield torch.tensor(x_batch), torch.flatten(torch.tensor(y_seq.iloc[batch_pos:batch_pos + batch_size].values))
        

In [27]:
# Train the model
n_total_steps = int(shuffled_y_train_data.shape[0] / train_batch_size)

for epoch in range(num_epochs):
    correct = 0
    for i, (x_batch, y_batch) in enumerate(chunker(shuffled_x_train_data, shuffled_y_train_data, sequence_length, train_batch_size)):  
        # origin shape: [N, 1, 28, 28]
        # resized: [N, 28, 28]
        y_batch = y_batch.to(device)
        x_batch = x_batch.to(device)
        # Forward pass
        outputs = model(x_batch)
        loss = criterion(outputs, y_batch)
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == y_batch).sum().item()
        if (i+1) % 10 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
    acc = 100.0 * correct / y_train_data.shape[0]
    print (f'Accuracy: {acc} %')
    # Test the model
    # In test phase, we don't need to compute gradients (for memory efficiency)
    correct = 0
    with torch.no_grad():
        correct = 0
        for i, (x_batch, y_batch) in enumerate(chunker(x_test_data, y_test_data, sequence_length, 5751)):  
            # origin shape: [N, 1, 28, 28]
            # resized: [N, 28, 28]
            y_batch = y_batch.to(device)
            x_batch = x_batch.to(device)
            outputs = model(x_batch)
            
            # max returns (value ,index)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == y_batch).sum().item()
        acc = 100.0 * correct / y_test_data.shape[0]
        print(f'Accuracy: {acc} %')

Epoch [1/10], Step [10/181], Loss: 1.7741
Epoch [1/10], Step [20/181], Loss: 1.7528
Epoch [1/10], Step [30/181], Loss: 1.7831
Epoch [1/10], Step [40/181], Loss: 1.7617
Epoch [1/10], Step [50/181], Loss: 1.7554
Epoch [1/10], Step [60/181], Loss: 1.7393
Epoch [1/10], Step [70/181], Loss: 1.7684
Epoch [1/10], Step [80/181], Loss: 1.7319
Epoch [1/10], Step [90/181], Loss: 1.7370
Epoch [1/10], Step [100/181], Loss: 1.7518
Epoch [1/10], Step [110/181], Loss: 1.7140
Epoch [1/10], Step [120/181], Loss: 1.7834
Epoch [1/10], Step [130/181], Loss: 1.7237
Epoch [1/10], Step [140/181], Loss: 1.7397
Epoch [1/10], Step [150/181], Loss: 1.7562
Epoch [1/10], Step [160/181], Loss: 1.6981
Epoch [1/10], Step [170/181], Loss: 1.7227
Epoch [1/10], Step [180/181], Loss: 1.6970
Accuracy: 21.91381215469613 %
Accuracy: 22.830812032689966 %
Epoch [2/10], Step [10/181], Loss: 1.7786
Epoch [2/10], Step [20/181], Loss: 1.7449
Epoch [2/10], Step [30/181], Loss: 1.7761
Epoch [2/10], Step [40/181], Loss: 1.7594


KeyboardInterrupt: 

In [59]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    for i, (x_batch, y_batch) in enumerate(chunker(x_test_data, y_test_data, sequence_length, 5751)):  
        # origin shape: [N, 1, 28, 28]
        # resized: [N, 28, 28]
        y_batch = y_batch.to(device)
        x_batch = x_batch.to(device)
        outputs = model(x_batch)
        
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == y_batch).sum().item()
    acc = 100.0 * correct / y_test_data.shape[0]
    print(f'Accuracy: {acc} %')

Accuracy of the network on the 10000 test images: 7.7030081724917405 %
